# Load dataset

In [ ]:
!git clone https://github.com/jsdabrowski/CAiSE-20

Cloning into 'CAiSE-20'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [ ]:
import pandas as pd

df_data = pd.read_excel('CAiSE-20/Ground_truth.xlsx')
df_data

,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated)
0,com.zentertain.photoeditor,gp:AOqpTOEW40L9WXqCjzq04bqaZImgMdzlczxIF3_ibs8...,1,May be i can check,NaN,NaN,NaN,NaN
1,com.zentertain.photoeditor,gp:AOqpTOF57AQPvmnCiWYurwLY-F2-mej25ON8RAFk-Ls...,1,It make me happy,NaN,NaN,NaN,NaN
2,com.zentertain.photoeditor,gp:AOqpTOHYdmt72q4tSD8TZ8A5fZQqGivlBkIbWuHuJMZ...,1,I have a normal phone and it made 1 of my pics...,NaN,pics,NaN,pics
3,com.zentertain.photoeditor,gp:AOqpTOFYnXMShrDJPS0jpM04pFQxYOJN1LDuX3lSNm0...,1,Love it so much,NaN,NaN,NaN,NaN
4,com.zentertain.photoeditor,gp:AOqpTOF_JO496wnThQ2kcYlPct_g1GhOmQyyVvHp4VV...,1,Cant get to install,NaN,install,NaN,install
...,...,...,...,...,...,...,...,...
2057,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,1,Every time I go on I can't browse or do anythi...,NaN,NaN,NaN,NaN
2058,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,2,I am fully connected to wifi and offline mode ...,NaN,offline mode,NaN,offline mode
2059,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,3,"Loved this app when it worked, can't use at al...",NaN,NaN,NaN,NaN
2060,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,4,Will cancel if not fixed very soon...,NaN,NaN,NaN,NaN


# Load repository and dependencies

In [ ]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-2jy17lpr
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-2jy17lpr
     |████████████████████████████████| 890kB 32.5MB/s 
     |████████████████████████████████| 901kB 46.3MB/s 
     |████████████████████████████████| 3.0MB 38.2MB/s 
     |████████████████████████████████| 1.2MB 34.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp37-none-any.whl size=101897 sha256=8a677aaf0f57895ed12d6f49ea1624f88acaff478d0cf3e79b54c65a43463df2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ixtlwj23/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
Successfully built sentence-transformers
Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
504MB [00:03, 135MB/s]
Archive:  distiluse-base-multilingual-cased.zip
   creating: la

# Generating word embedding

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

np.set_printoptions(threshold=100)
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO,handlers=[LoggingHandler()])

language_model = SentenceTransformer('./language_model')

2021-06-07 23:21:41 - Load pretrained SentenceTransformer: ./language_model
2021-06-07 23:21:41 - Load SentenceTransformer from folder: ./language_model
2021-06-07 23:21:46 - Use pytorch device: cpu


In [ ]:
pd.DataFrame(df_data['App id'].unique())

,0
0,com.zentertain.photoeditor
1,B004LOMB2Q
2,B004SIIBGU
3,com.whatsapp
4,B005ZXWMUS
5,com.twitter.android
6,B0094BB4TW
7,com.spotify.music


In [ ]:
APP_NAME = 'B004LOMB2Q'
RUN=3

In [ ]:
sentence_embedding_cache = {}
requirement_embedding_cache = {}
L_sentence = []
L_requirement = []
for index,row in df_data.iterrows():
  L_sentence.append(row['Sentence content'])
  try:
    for r in row['Feature (All Annotated)'].split(';'):
      #print(r.strip())
      L_requirement.append(r.strip())
  except:
    1

L_sentence_embedding = list(language_model.encode(L_sentence))
L_requirement_embedding = list(language_model.encode(L_requirement))



In [ ]:
for i in range(0,len(L_sentence)):
  sentence_embedding_cache[L_sentence[i]] = L_sentence_embedding[i]

for i in range(0,len(L_requirement)):
  requirement_embedding_cache[L_requirement[i]] = L_requirement_embedding[i]

In [ ]:
df_app = df_data[df_data['App id']==APP_NAME]
L = []
for index,row in df_app.iterrows():
  L.append(sentence_embedding_cache[row['Sentence content']])
df_app['sentence_embedding'] = L
df_app

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated),sentence_embedding
154,B004LOMB2Q,A3NOCGVOLK1IPO,1,I've used Evernote for quite awhile to take no...,NaN,take notes;share;edit,NaN,take notes;share;edit,"[0.0364771, 0.02155217, -0.06920058, -0.023209..."
155,B004LOMB2Q,A3NOCGVOLK1IPO,2,"Not as nice as Microsoft OneNote, but at least...",NaN,NaN,NaN,NaN,"[-0.04071294, -0.034614015, 0.04678943, -0.049..."
156,B004LOMB2Q,A3NOCGVOLK1IPO,3,Nice to see that it runs on the Kindle Fire HD.,NaN,NaN,NaN,NaN,"[0.027171684, 0.043886445, 0.028232709, 0.0127..."
157,B004LOMB2Q,ALYCIMJ6ZVBA5,1,Love that I can sync to my other devices.,sync to my other devices,NaN,NaN,sync to my other devices,"[0.048731025, -0.037876833, -0.052123584, -0.0..."
158,B004LOMB2Q,ALYCIMJ6ZVBA5,2,It has been a big help in keeping me organized...,NaN,keeping me organized with my lists,NaN,keeping me organized with my lists,"[0.0043796827, 0.070137784, 0.017053999, 0.050..."
...,...,...,...,...,...,...,...,...,...
516,B004LOMB2Q,A3AJ6LK7A5ZZ9R,4,I like the way you can organize into notebooks.,organize into notebooks,NaN,NaN,organize into notebooks,"[0.030230004, -0.0257629, 0.04704534, 0.014672..."
517,B004LOMB2Q,A3AJ6LK7A5ZZ9R,5,everything is very useful.,NaN,NaN,NaN,NaN,"[0.09980962, 0.023971433, -0.0021783903, -0.03..."
518,B004LOMB2Q,A3AJ6LK7A5ZZ9R,6,I like it a lot.,NaN,NaN,NaN,NaN,"[0.08920757, -0.07764688, -0.013945842, -0.052..."
519,B004LOMB2Q,AJCBQCE0DPTGX,1,great app for taking notes across all platforms.,taking notes,NaN,NaN,taking notes,"[0.016192105, -0.049758844, -0.035583925, -0.0..."


# Generating Clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples
import numpy as np
import math

def get_kmeans(df_):

  clustering = {}
  df_temp = df_.copy()
  X = np.array(df_temp['requirement_embedding'].to_list())
  num_clusters = int(math.sqrt(len(df_temp)))
  if num_clusters < 2:
    print('Error! Dataset size!')
    return
  if num_clusters >= 30:
    num_clusters = 30

  cluster_parent = df_temp['cluster'].unique()[0]
  print('Running cluster from ',cluster_parent,' total=',len(X),' num_clusters=',num_clusters)
  kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=RUN, init='random',verbose=0,batch_size=1000,max_no_improvement=1000).fit(X)
  df_temp['cluster_parent'] = cluster_parent
  df_temp['cluster'] = np.array(kmeans.labels_)+(10*cluster_parent)
  print('Cluster parent',df_temp['cluster_parent'].unique())
  print('Cluster ',df_temp['cluster'].unique())
  clustering[cluster_parent] = [df_temp,kmeans]

  return df_temp, clustering

In [ ]:
requirements = set()
df_temp = df_data[df_data['App id']==APP_NAME]
for index,row in df_temp[['Feature (All Annotated)']].dropna().iterrows():
    for r in row['Feature (All Annotated)'].split(';'):
      requirements.add(r.strip())

df_requirements = pd.DataFrame(requirements)
df_requirements.columns =  ['requirement']
L = []
for r in requirements:
    L.append(requirement_embedding_cache[r])
df_requirements['requirement_embedding'] = L
df_requirements

2021-06-07 23:23:21 - NumExpr defaulting to 2 threads.


,requirement,requirement_embedding
0,find,"[0.084729455, -0.0035362628, 0.025818553, 0.00..."
1,saved,"[0.05206348, 0.019318992, -0.0911457, 0.005839..."
2,create your account,"[-0.04271362, -0.037511367, -0.03134732, 0.041..."
3,task,"[0.059522662, 0.06452647, 0.011401685, 0.02569..."
4,information handling capabilities,"[0.0012853846, -0.00053734303, 0.010118408, 0...."
...,...,...
254,organize my class notes,"[-0.030215815, 0.0068719573, 0.01923871, 0.032..."
255,separate notebooks,"[-0.009750171, 0.0013386105, -0.033240546, 0.0..."
256,sync,"[0.04565755, 0.0011984614, -0.082250625, -0.00..."
257,add-in,"[-0.023695182, -0.012073642, -0.03556952, 0.00..."


In [ ]:
stack = []
df_temp = df_requirements.copy()
df_temp['cluster_parent'] = 0
df_temp['cluster'] = 1
stack.append(df_temp)

hc = []
clustering = []

while(len(stack)!=0):
  df_temp = stack.pop()
  if len(df_temp) <= 20: continue
  df_temp,clust = get_kmeans(df_temp)
  clustering.append(clust)
  hc.append(df_temp)
  for cluster_ in df_temp.cluster.unique():
    stack.append(df_temp[df_temp.cluster==cluster_])

Running cluster from  1  total= 259  num_clusters= 16
Cluster parent [1]
Cluster  [15 13 16 17 23 11 18 21 10 22 25 24 12 19 14 20]
Running cluster from  22  total= 56  num_clusters= 7
Cluster parent [22]
Cluster  [221 222 224 225 226 220 223]
Running cluster from  221  total= 25  num_clusters= 5
Cluster parent [221]
Cluster  [2212 2214 2210 2211 2213]
Running cluster from  18  total= 25  num_clusters= 5
Cluster parent [18]
Cluster  [182 184 180 181 183]
Running cluster from  17  total= 30  num_clusters= 5
Cluster parent [17]
Cluster  [172 174 173 170 171]
Running cluster from  16  total= 22  num_clusters= 4
Cluster parent [16]
Cluster  [163 161 160 162]


In [ ]:
df_hc = pd.concat(hc)
df_hc

,requirement,requirement_embedding,cluster_parent,cluster
0,find,"[0.084729455, -0.0035362628, 0.025818553, 0.00...",1,15
1,saved,"[0.05206348, 0.019318992, -0.0911457, 0.005839...",1,13
2,create your account,"[-0.04271362, -0.037511367, -0.03134732, 0.041...",1,16
3,task,"[0.059522662, 0.06452647, 0.011401685, 0.02569...",1,17
4,information handling capabilities,"[0.0012853846, -0.00053734303, 0.010118408, 0....",1,23
...,...,...,...,...
176,dictate 500 words,"[0.051059432, 0.011996878, -0.010506158, 0.085...",16,163
200,use it for school assignments,"[0.050459515, -0.008653242, -9.8246615e-05, -0...",16,161
219,make a file,"[0.0037980028, -0.0011163005, -0.080420844, 0....",16,161
222,format it with tables,"[-0.017584639, -0.022017432, -0.0042687263, 0....",16,162


# Centroid Labeling (Baseline)

In [ ]:
def centroid_labeling(df_hc):

  cluster_labels = {}

  df_hc1 = df_hc.copy() 
  for cluster in df_hc1.cluster.unique():
    cluster_centroid = np.mean(np.array(df_hc1[df_hc1.cluster==cluster].requirement_embedding.to_list()),axis=0)
    min_dist = 999999999
    cluster_label = None

    for index,row in df_hc1[df_hc1.cluster==cluster].iterrows():
      requirement_emb = row['requirement_embedding']
      dist = np.linalg.norm(requirement_emb-cluster_centroid)
      if dist < min_dist:
        cluster_label = row['requirement']
        min_dist = dist
    cluster_labels[cluster]=cluster_label

  L = []
  for index,row in df_hc1.iterrows():
    L.append(cluster_labels[row['cluster']])

  df_hc1['cluster_label'] = L
  return df_hc1

df_hc1 = centroid_labeling(df_hc)
df_hc1

,requirement,requirement_embedding,cluster_parent,cluster,cluster_label
0,add web,"[-0.035125732, 0.02539581, -0.03774613, 0.0031...",1,23,attachments
1,Installed,"[-0.014426949, 0.0056241737, -0.025648205, 0.0...",1,16,jot down ideas
2,email myself links,"[-0.025898542, 0.02763646, -0.0081680035, 0.02...",1,17,email myself notes
3,write a note,"[-0.021114131, 0.021097096, -0.097701244, 0.05...",1,11,take notes
4,clip selected text,"[0.018398028, 0.09764467, -0.05678857, 0.08727...",1,23,attachments
...,...,...,...,...,...
211,add-ons,"[-0.03760511, -0.012892555, -0.041193653, 0.01...",23,233,attachments
228,PDFs are shown as attachments,"[0.02998425, -0.03281471, 0.006808938, 0.01948...",23,233,attachments
241,tags web content,"[-0.08101787, 0.04761891, -0.057284478, 0.0147...",23,232,webpages
245,attach button,"[-0.051455855, -0.0388197, -0.0011252412, 0.04...",23,233,attachments


# Baseline Evaluation

In [ ]:
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics import f1_score
from sklearn.metrics.cluster import normalized_mutual_info_score



def isNaN(num):
  return num != num


def eval_labeling(df_hc_,df_app):

  
  results = []

  for cluster_parent in df_hc_.cluster_parent.unique():

    L = []

    for index,row in df_app.iterrows():
      for r in df_hc_[df_hc_.cluster_parent==cluster_parent].requirement.unique():
        if isNaN(row['Feature (All Annotated)'])==False:
          if r in row['Feature (All Annotated)']:
            L.append(row.to_dict())

    df_app2 = pd.DataFrame(L)

    cluster_reference = []
    cluster_labeling = []
    
    for index,row in df_app2.iterrows():

      ####### REFERENCE CLUSTER #######
      cluster_sel = -1
      min_dist = 999999999
      sentence_embedding = row['sentence_embedding']

      df_temp = df_hc_[df_hc_.cluster_parent==cluster_parent]
      for cluster in df_temp.cluster.unique():
        cluster_centroid = np.mean(np.array(df_temp[df_temp.cluster==cluster].requirement_embedding.to_list()),axis=0)
        dist = np.linalg.norm(sentence_embedding-cluster_centroid)
        if dist < min_dist:
          cluster_sel = cluster
          min_dist = dist
      
      cluster_reference.append(cluster_sel)


      ####### LABELING CLUSTER #######
      cluster_sel = -1
      min_dist = 999999999
      sentence_embedding = row['sentence_embedding']

      df_temp = df_hc_[df_hc_.cluster_parent==cluster_parent]
      for cluster in df_temp.cluster.unique():
        cluster_centroid = requirement_embedding_cache[df_temp[df_temp.cluster==cluster].head(1).cluster_label.values[0]]
        dist = np.linalg.norm(sentence_embedding-cluster_centroid)
        if dist < min_dist:
          cluster_sel = cluster
          min_dist = dist
      
      cluster_labeling.append(cluster_sel)

    # print(cluster_parent)
    # print(cluster_reference)
    # print(cluster_labeling)
    # print(v_measure_score(cluster_reference,cluster_labeling))
    # print(f1_score(cluster_reference,cluster_labeling,average='micro'))
    # print('--------------')
    results.append((cluster_parent,len(cluster_reference),v_measure_score(cluster_reference,cluster_labeling),f1_score(cluster_reference,cluster_labeling,average='micro')))

  return results

In [ ]:
results_baseline = eval_labeling(df_hc1,df_app)
print(results_baseline)

[(1, 320, 0.5757110957213297, 0.540625), (12, 40, 0.45817406455082094, 0.7), (17, 83, 0.4977118955747743, 0.6265060240963856), (20, 32, 0.2758659990016619, 0.46875)]


# Proposed Cluster Labeling

In [ ]:
df_app = df_data[df_data['App id']==APP_NAME]
L = []
for index,row in df_app.iterrows():
  L.append(sentence_embedding_cache[row['Sentence content']])
df_app['sentence_embedding'] = L
df_app

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated),sentence_embedding
154,B004LOMB2Q,A3NOCGVOLK1IPO,1,I've used Evernote for quite awhile to take no...,NaN,take notes;share;edit,NaN,take notes;share;edit,"[0.0364771, 0.02155217, -0.06920058, -0.023209..."
155,B004LOMB2Q,A3NOCGVOLK1IPO,2,"Not as nice as Microsoft OneNote, but at least...",NaN,NaN,NaN,NaN,"[-0.04071294, -0.034614015, 0.04678943, -0.049..."
156,B004LOMB2Q,A3NOCGVOLK1IPO,3,Nice to see that it runs on the Kindle Fire HD.,NaN,NaN,NaN,NaN,"[0.027171684, 0.043886445, 0.028232709, 0.0127..."
157,B004LOMB2Q,ALYCIMJ6ZVBA5,1,Love that I can sync to my other devices.,sync to my other devices,NaN,NaN,sync to my other devices,"[0.048731025, -0.037876833, -0.052123584, -0.0..."
158,B004LOMB2Q,ALYCIMJ6ZVBA5,2,It has been a big help in keeping me organized...,NaN,keeping me organized with my lists,NaN,keeping me organized with my lists,"[0.0043796827, 0.070137784, 0.017053999, 0.050..."
...,...,...,...,...,...,...,...,...,...
516,B004LOMB2Q,A3AJ6LK7A5ZZ9R,4,I like the way you can organize into notebooks.,organize into notebooks,NaN,NaN,organize into notebooks,"[0.030230004, -0.0257629, 0.04704534, 0.014672..."
517,B004LOMB2Q,A3AJ6LK7A5ZZ9R,5,everything is very useful.,NaN,NaN,NaN,NaN,"[0.09980962, 0.023971433, -0.0021783903, -0.03..."
518,B004LOMB2Q,A3AJ6LK7A5ZZ9R,6,I like it a lot.,NaN,NaN,NaN,NaN,"[0.08920757, -0.07764688, -0.013945842, -0.052..."
519,B004LOMB2Q,AJCBQCE0DPTGX,1,great app for taking notes across all platforms.,taking notes,NaN,NaN,taking notes,"[0.016192105, -0.049758844, -0.035583925, -0.0..."


In [ ]:
def get_contextual_embedding(requirement,df_app,alpha=0.9,show_progress_bar=False):

  df_feature = df_app[['Sentence content','Feature (All Annotated)','sentence_embedding']].dropna()
  df_req_temp = df_feature[df_feature['Feature (All Annotated)'].str.contains(requirement)]
  context_center = np.mean(np.array(df_req_temp['sentence_embedding'].to_list()),axis=0)
  requirement_emb = requirement_embedding_cache[requirement]
  contextual_embedding = (1.0-alpha)*requirement_emb + (alpha)*context_center

  return contextual_embedding

#get_contextual_embedding('premium',df_app,alpha=0.5)

In [ ]:
def contextual_cluster_labeling(df_hc,df_app,alpha=0.1):

  cluster_labels = {}
  cache_used_labels = set()

  df_hc2 = df_hc.copy() 
  for cluster in df_hc2.cluster.unique():
    cluster_centroid = np.mean(np.array(df_hc2[df_hc2.cluster==cluster].requirement_embedding.to_list()),axis=0)
    min_dist = 999999999
    cluster_label = None

    for index,row in df_hc2[df_hc2.cluster==cluster].iterrows():
      requirement_emb = get_contextual_embedding(row['requirement'],df_app,alpha=alpha)
      #requirement_emb = row['requirement_embedding']
      dist = np.linalg.norm(requirement_emb-cluster_centroid)
      if dist < min_dist and row['requirement'] not in cache_used_labels:
        cluster_label = row['requirement']
        min_dist = dist
    cluster_labels[cluster]=cluster_label
    cache_used_labels.add(cluster_label)

  L = []
  for index,row in df_hc2.iterrows():
    L.append(cluster_labels[row['cluster']])

  df_hc2['cluster_label'] = L
  return df_hc2



In [ ]:
from tqdm.notebook import tqdm
results_proposal = {}
for alpha in tqdm(np.arange (0, 1.02, 0.02)):
  
  df_hc2 = contextual_cluster_labeling(df_hc,df_app,alpha=alpha)
  results_proposal[alpha] = eval_labeling(df_hc2,df_app)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/d

# Saving results

In [ ]:
res1 = pd.DataFrame(results_baseline)
res1['method'] = 'baseline'
res1.columns = ['hie_level','total','v_measure','f1','method']


L = []
for alpha in results_proposal:
  for item in results_proposal[alpha]:
    R = list(item)
    R.append(alpha)
    L.append(R)



res2 = pd.DataFrame(L)
res2['method'] = 'proposal'
res2.columns = ['hie_level','total','v_measure','f1','alpha','method']

res_final = pd.concat([res1,res2])
#res_final.columns = ['hie_level','total','v_measure','f1','alpha','method']


res_final

,hie_level,total,v_measure,f1,method,alpha
0,1,320,0.575711,0.540625,baseline,NaN
1,12,40,0.458174,0.700000,baseline,NaN
2,17,83,0.497712,0.626506,baseline,NaN
3,20,32,0.275866,0.468750,baseline,NaN
0,1,320,0.575711,0.540625,proposal,0.00
...,...,...,...,...,...,...
199,20,32,0.242936,0.375000,proposal,0.98
200,1,320,0.541171,0.496875,proposal,1.00
201,12,40,0.378105,0.600000,proposal,1.00
202,17,83,0.507742,0.518072,proposal,1.00


In [ ]:
res_final.to_excel(APP_NAME+'_run_'+str(RUN)+'_.xls')

In [ ]:
res_final.groupby(['hie_level','method']).max()

total  v_measure        f1  alpha
hie_level method                                     
1         baseline    320   0.575711  0.540625    NaN
          proposal    320   0.598339  0.540625    1.0
12        baseline     40   0.458174  0.700000    NaN
          proposal     40   0.458174  0.700000    1.0
17        baseline     83   0.497712  0.626506    NaN
          proposal     83   0.532654  0.626506    1.0
20        baseline     32   0.275866  0.468750    NaN
          proposal     32   0.333646  0.625000    1.0

# Assessment

In [ ]:
import pandas as pd

df_data = pd.read_csv('_all_experiments.xlsx - all_run.csv')
df_data

,Unnamed: 0,app,run,method,alpha,hie_level,hie_level2,total,v_measure,f1
0,0,Evernote,1,baseline,-,1,1,512,"0,5404120043","0,529296875"
1,1,Evernote,1,baseline,-,18,2,69,"0,6098795308","0,6811594203"
2,2,Evernote,1,baseline,-,20,2,188,"0,4719286616","0,4946808511"
3,3,Evernote,1,baseline,-,205,3,31,"0,1331786174","0,3870967742"
4,4,Evernote,1,baseline,-,25,2,97,"0,4783016984","0,4639175258"
...,...,...,...,...,...,...,...,...,...,...
14971,307,eBay,9,proposal,1,17,2,26,"0,4517717521","0,5769230769"
14972,308,eBay,9,proposal,1,13,2,23,"0,4037120994","0,652173913"
14973,309,eBay,9,proposal,1,11,2,95,"0,2969004485","0,3263157895"
14974,310,eBay,9,proposal,1,112,3,71,"0,1429765828","0,05633802817"


In [ ]:
L = []
for index,row in df_data.iterrows():
  L.append(float(row['f1'].replace(',','.')))


df_data['f1'] = L
df_data

,Unnamed: 0,app,run,method,alpha,hie_level,hie_level2,total,v_measure,f1
0,0,Evernote,1,baseline,-,1,1,512,"0,5404120043",0.529297
1,1,Evernote,1,baseline,-,18,2,69,"0,6098795308",0.681159
2,2,Evernote,1,baseline,-,20,2,188,"0,4719286616",0.494681
3,3,Evernote,1,baseline,-,205,3,31,"0,1331786174",0.387097
4,4,Evernote,1,baseline,-,25,2,97,"0,4783016984",0.463918
...,...,...,...,...,...,...,...,...,...,...
14971,307,eBay,9,proposal,1,17,2,26,"0,4517717521",0.576923
14972,308,eBay,9,proposal,1,13,2,23,"0,4037120994",0.652174
14973,309,eBay,9,proposal,1,11,2,95,"0,2969004485",0.326316
14974,310,eBay,9,proposal,1,112,3,71,"0,1429765828",0.056338


In [ ]:
def eval_hie_f1(df_data,run,app,method,alpha):
  df_temp = df_data[df_data.method==method]
  df_temp = df_temp[df_temp.app==app]
  df_temp = df_temp[df_temp.run==run]
  df_temp = df_temp[df_temp.alpha==alpha]
  total = 0
  f1_sum = 0
  for index,row in df_temp.iterrows():
    f1_sum += row['f1']*row['total']
    total += row['total']
  return f1_sum/total

In [ ]:
eval_hie_f1(df_data,1,'eBay','baseline','-')

0.682105263143579

In [ ]:
for alpha in df_data[df_data.method=='proposal'].alpha.unique():
  f1 = eval_hie_f1(df_data,1,'eBay','proposal',alpha)
  print(alpha,f1)

0 0.682105263143579
0,02 0.6757894736705263
0,04 0.6652631579105264
0,06 0.6652631579105264
0,08 0.6568421052905263
0,1 0.6589473684463159
0,12 0.6589473684463159
0,14 0.6589473684463159
0,16 0.6589473684463159
0,18 0.6652631579263159
0,2 0.6652631579263159
0,22 0.6631578947263158
0,24 0.6800000000214736
0,26 0.6800000000214736
0,28 0.6673684210753683
0,3 0.6673684210753683
0,32 0.6673684210753683
0,34 0.6673684210753683
0,36 0.6673684210753683
0,38 0.6757894737098946
0,4 0.6778947368656841
0,42 0.6778947368656841
0,44 0.6800000000214736
0,46 0.6884210526524209
0,48 0.6863157894966316
0,5 0.6863157894966316
0,52 0.6863157894966316
0,54 0.6863157894966316
0,56 0.6863157894966316
0,58 0.6863157894966316
0,6 0.6884210526511579
0,62 0.6884210526511579
0,64 0.680000000031158
0,66 0.680000000031158
0,68 0.6610526316037895
0,7 0.6610526316037895
0,72 0.6715789473637895
0,74 0.6736842105037896
0,76 0.671578947348
0,78 0.6736842105480001
0,8 0.6800000000280001
0,82 0.6800000000280001
0,84 0.680

In [ ]:
L = []
df_temp = df_data[df_data.method=='proposal']
for run in df_temp.run.unique():
  for app in df_temp.app.unique():
    for alpha in df_temp.alpha.unique():
      f1 = eval_hie_f1(df_data,run,app,'proposal',alpha)
      L.append([run,app,alpha,f1])

df_results = pd.DataFrame(L)
df_results.columns = ['run','app','alpha','f1']
df_results

,run,app,alpha,f1
0,1,Evernote,0,0.525188
1,1,Evernote,"0,02",0.533762
2,1,Evernote,"0,04",0.533762
3,1,Evernote,"0,06",0.534834
4,1,Evernote,"0,08",0.534834
...,...,...,...,...
4075,9,eBay,"0,92",0.460076
4076,9,eBay,"0,94",0.460076
4077,9,eBay,"0,96",0.450570
4078,9,eBay,"0,98",0.448669


In [ ]:
df_results.to_excel('results_f1_hie.xls')

# Generating View

In [ ]:
df_app

,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated),sentence_embedding
154,B004LOMB2Q,A3NOCGVOLK1IPO,1,I've used Evernote for quite awhile to take no...,NaN,take notes;share;edit,NaN,take notes;share;edit,"[0.0364771, 0.02155217, -0.06920058, -0.023209..."
155,B004LOMB2Q,A3NOCGVOLK1IPO,2,"Not as nice as Microsoft OneNote, but at least...",NaN,NaN,NaN,NaN,"[-0.04071294, -0.034614015, 0.04678943, -0.049..."
156,B004LOMB2Q,A3NOCGVOLK1IPO,3,Nice to see that it runs on the Kindle Fire HD.,NaN,NaN,NaN,NaN,"[0.027171684, 0.043886445, 0.028232709, 0.0127..."
157,B004LOMB2Q,ALYCIMJ6ZVBA5,1,Love that I can sync to my other devices.,sync to my other devices,NaN,NaN,sync to my other devices,"[0.048731025, -0.037876833, -0.052123584, -0.0..."
158,B004LOMB2Q,ALYCIMJ6ZVBA5,2,It has been a big help in keeping me organized...,NaN,keeping me organized with my lists,NaN,keeping me organized with my lists,"[0.0043796827, 0.070137784, 0.017053999, 0.050..."
...,...,...,...,...,...,...,...,...,...
516,B004LOMB2Q,A3AJ6LK7A5ZZ9R,4,I like the way you can organize into notebooks.,organize into notebooks,NaN,NaN,organize into notebooks,"[0.030230004, -0.0257629, 0.04704534, 0.014672..."
517,B004LOMB2Q,A3AJ6LK7A5ZZ9R,5,everything is very useful.,NaN,NaN,NaN,NaN,"[0.09980962, 0.023971433, -0.0021783903, -0.03..."
518,B004LOMB2Q,A3AJ6LK7A5ZZ9R,6,I like it a lot.,NaN,NaN,NaN,NaN,"[0.08920757, -0.07764688, -0.013945842, -0.052..."
519,B004LOMB2Q,AJCBQCE0DPTGX,1,great app for taking notes across all platforms.,taking notes,NaN,NaN,taking notes,"[0.016192105, -0.049758844, -0.035583925, -0.0..."


In [ ]:
df_hc2 = contextual_cluster_labeling(df_hc,df_app,alpha=0.1)

In [ ]:
df_hc2

,requirement,requirement_embedding,cluster_parent,cluster,cluster_label
0,find,"[0.084729455, -0.0035362628, 0.025818553, 0.00...",1,15,check box
1,saved,"[0.05206348, 0.019318992, -0.0911457, 0.005839...",1,13,take pictures
2,create your account,"[-0.04271362, -0.037511367, -0.03134732, 0.041...",1,16,jot down ideas
3,task,"[0.059522662, 0.06452647, 0.011401685, 0.02569...",1,17,capture
4,information handling capabilities,"[0.0012853846, -0.00053734303, 0.010118408, 0....",1,23,updating
...,...,...,...,...,...
176,dictate 500 words,"[0.051059432, 0.011996878, -0.010506158, 0.085...",16,163,reset the password
200,use it for school assignments,"[0.050459515, -0.008653242, -9.8246615e-05, -0...",16,161,write down recipes
219,make a file,"[0.0037980028, -0.0011163005, -0.080420844, 0....",16,161,write down recipes
222,format it with tables,"[-0.017584639, -0.022017432, -0.0042687263, 0....",16,162,put all the slides in one column


In [ ]:
df_hie = df_hc2[['cluster_parent','cluster','cluster_label']].drop_duplicates()
df_hie

,cluster_parent,cluster,cluster_label
0,1,15,check box
1,1,13,take pictures
2,1,16,jot down ideas
3,1,17,capture
4,1,23,updating
5,1,11,access notes
6,1,18,syncs
7,1,21,sharing
8,1,10,save them I specific folders
15,1,22,take notes


In [ ]:
def get_label(df_hie,node_id):
  if node_id==1: return 'Root'
  else:
    return df_hie[df_hie.cluster==node_id].cluster_label.to_list()[0]

def visitar(df_hie,node):
  #print('Visitando: ',node,get_label(df_hie,node))
  tab = ""
  for i in range(0,len(str(node))): tab+="\t"
  print(tab+'{"name" : "'+get_label(df_hie,node)+'", "children" : [');
  df_node = df_hie[df_hie.cluster_parent==node]
  if len(df_node) > 0:
    for index,row in df_node.iterrows():
      visitar(df_hie,row['cluster'])
      #print('Adding ',row['cluster'], get_label(df_hie,row['cluster']))
  print(tab+'] },')
visitar(df_hie,1)

	{"name" : "Root", "children" : [
		{"name" : "check box", "children" : [
		] },
		{"name" : "take pictures", "children" : [
		] },
		{"name" : "jot down ideas", "children" : [
			{"name" : "reset the password", "children" : [
			] },
			{"name" : "write down recipes", "children" : [
			] },
			{"name" : "jotting down ideas", "children" : [
			] },
			{"name" : "put all the slides in one column", "children" : [
			] },
		] },
		{"name" : "capture", "children" : [
			{"name" : "sign", "children" : [
			] },
			{"name" : "scan", "children" : [
			] },
			{"name" : "premium", "children" : [
			] },
			{"name" : "review", "children" : [
			] },
			{"name" : "register", "children" : [
			] },
		] },
		{"name" : "updating", "children" : [
		] },
		{"name" : "access notes", "children" : [
		] },
		{"name" : "syncs", "children" : [
			{"name" : "syncs with all my devices", "children" : [
			] },
			{"name" : "synchronizing across platforms", "children" : [
			] },
			{"name" : "sync", "childre

# View Cluster

In [ ]:
data_sel = None
for item in clustering:
  for parent in item:
    print(parent)
    if parent==1: data_sel = item
  #break

1
22
221
18
17
16


In [ ]:
data_sel[1][0].sort_values(by='cluster').tail(50)

,requirement,requirement_embedding,cluster_parent,cluster
40,make quick note,"[-0.03275299, -0.028174011, -0.063087545, 0.03...",1,22
93,note-taking,"[-0.02006421, -0.019334538, -0.099763066, 0.03...",1,22
50,modify your notes,"[-0.038745236, 0.03895115, -0.07445832, 0.0598...",1,22
146,keep picture notes,"[0.0063220426, -0.014439564, -0.08473054, 0.08...",1,22
147,wrote that note down,"[-0.014184688, 0.023298306, -0.058327705, 0.04...",1,22
77,take my notes,"[-0.0028481546, 0.006901072, -0.10608642, 0.06...",1,22
181,taking text notes,"[-0.05418444, 0.029422924, -0.08922365, 0.0654...",1,22
173,keep track of all my notes,"[0.026762001, 0.00037281585, -0.06747962, 0.03...",1,22
170,save notes,"[0.045533095, 0.034110505, -0.11593404, 0.0524...",1,22
169,add notes,"[-0.044961166, 0.029017838, -0.070679665, 0.04...",1,22
